In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import string 
import seaborn as sns 
import nltk 
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import os 
import contractions 

## Findings from Exploratory Data Analysis 

#### 1. Statistically its proven that The number of words in Transcript is different for Selection and Rejection Candidates (Using T-Test) 

#### 2. Using the Tf-IDF vectorizer , it cames out that only similarity between transcript and Job decription is higher for selected candidates compared with similarity between transcript and resume , resume and Job description 

#### 3. But when we used the Word to Vector model, the similarity between Resume and Job description, Job description and Transcript , Transcript and Resume are higher for selected candidates compared with the rejected candidates

#### 4. data analyst,data engineer,data scientist,product manager,software engineer,ui designer,ui engineer  are the only roles which have only enough data to train on all other roles have data between 10-60 data points, so we dropped them 


In [61]:
path = "D:/InterviewAutomation/data"

In [63]:
excel_files = [file for file in os.listdir(path) if file.endswith(('.xls', '.xlsx'))]

In [64]:
for file in excel_files:
    df = pd.read_excel(path+'/'+file)
    print(df.columns)

Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript'

In [5]:
# # Changing the Columns of the 4th dataset as it does have the Wrong Columns
# df = pd.read_excel(excel_files[3])
# df.columns = ['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
#        'Reason for decision', 'Job Description']
# df.to_excel(path+'/'+'dataset4.xlsx',index=False)

In [6]:
# for file in excel_files:
#     df = pd.read_excel(path+'/'+file)
#     print(df.columns)

In [65]:
# Code for Merging all the datasets and creating the full dataset for traing and testing
dataframes = []

for file in excel_files:
    file_path = os.path.join(path, file)
    df = pd.read_excel(file_path)
    dataframes.append(df)

# Combine all dataframes into a single dataframe
data = pd.concat(dataframes, ignore_index=True)

In [66]:
data.shape

(3250, 8)

In [67]:
data.head()

,ID,Name,Role,Transcript,Resume,decision,Reason for decision,Job Description
0,brenbr359,Brent Brown,Product Manager,Product Manager Interview Transcript\n\nInterv...,Here's a sample resume for Brent Brown applyin...,select,Experience,We are looking for a skilled Product Manager w...
1,jameay305,James Ayala,Software Engineer,Software Engineer Interview Transcript\n\nInte...,Here's a sample resume for James Ayala applyin...,select,Experience,We are looking for a skilled Software Engineer...
2,scotri565,Scott Rivera,Data Engineer,Here is a simulated interview for Scott Rivera...,Here's a sample resume for Scott Rivera applyi...,reject,Experience,We are looking for a skilled Data Engineer wit...
3,emilke232,Emily Kelly,UI Engineer,Interview Transcript: Emily Kelly for UI Engin...,Here's a sample resume for Emily Kelly:\n\nEmi...,select,Experience,We are looking for a skilled UI Engineer with ...
4,ashlra638,Ashley Ray,Data Scientist,Data Scientist Interview Transcript\n\nCompany...,Here's a sample resume for Ashley Ray applying...,reject,Cultural Fit,We are looking for a skilled Data Scientist wi...


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3250 entries, 0 to 3249
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   3250 non-null   object
 1   Name                 3250 non-null   object
 2   Role                 3250 non-null   object
 3   Transcript           3250 non-null   object
 4   Resume               3250 non-null   object
 5   decision             3250 non-null   object
 6   Reason for decision  3250 non-null   object
 7   Job Description      3250 non-null   object
dtypes: object(8)
memory usage: 203.2+ KB


In [69]:
for col in data.columns:
    data[col] = data[col].apply(lambda x: x.lower())

In [70]:
data['decision'].unique()

array(['select', 'reject', 'rejected', 'selected'], dtype=object)

In [71]:
def process_decision(text):
    if text in ['select','selected']:
        return 'select'
    else :
        return 'reject'

In [72]:
data['decision'] = data['decision'].apply(process_decision )

In [73]:
data['Role'].unique()

array(['product manager', 'software engineer', 'data engineer',
       'ui engineer', 'data scientist', 'data analyst', 'ui designer',
       'digital marketing specialist', 'cybersecurity specialist',
       'hr specialist', 'network engineer', 'graphic designer',
       'game developer', 'mobile app developer', 'cloud architect',
       'business analyst', 'database administrator', 'devops engineer',
       'machine learning engineer', 'content writer',
       'system administrator', 'ui/ux designer', 'ai engineer',
       'project manager', 'software developer'], dtype=object)

In [74]:
unique_count = data.groupby('Role')['ID'].count()
unique_count

Role
ai engineer                      13
business analyst                 11
cloud architect                   8
content writer                    5
cybersecurity specialist          9
data analyst                    267
data engineer                   509
data scientist                  592
database administrator            9
devops engineer                  16
digital marketing specialist     14
game developer                   10
graphic designer                 15
hr specialist                    12
machine learning engineer        13
mobile app developer              7
network engineer                 16
product manager                 514
project manager                  56
software developer               66
software engineer               551
system administrator              5
ui designer                     234
ui engineer                     291
ui/ux designer                    7
Name: ID, dtype: int64

In [75]:
## Dropping Rows which have count < 100 
for count , role in zip(unique_count.values, unique_count.index):
    if count < 100 :
        data = data[data['Role'] != role] 

In [76]:
unique_count = data.groupby('Role')['ID'].count()
unique_count

Role
data analyst         267
data engineer        509
data scientist       592
product manager      514
software engineer    551
ui designer          234
ui engineer          291
Name: ID, dtype: int64

In [77]:
data.shape

(2958, 8)

In [78]:
## Performing Feature Engineering
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [81]:
nlp = spacy.load("en_core_web_sm") 
stopword_s = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

def textpreprocessing(text):
    text = contractions.fix(text) # expanding the contractions like , I'm with I am and it's with It is and so on...
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_`{|}~]', '', text)  # Removing all these characters from the text 
    sentences = re.split(r'(?<=[.!?])\s+', text)   # Splitting the text as sentences using the regular expressions     
    for idx, sent in enumerate(sentences):
        words = nlp(sent)  # converting the sentence into words
        words = [word.text for word in words if word.text.lower() not in stopword_s] # Removing the stopwords from the text
        words = [lemmatizer.lemmatize(word) for word in words]  # Using the Lemmatization techniques
        sentences[idx] = ' '.join(words)  # Replacing the original sentence with the new sentence 
    return ' '.join(sentences)  # Join the sentences back together  

In [83]:
data['Transcript_processed'] = data['Transcript'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.xlsx')

In [84]:
data['Resume_processed'] = data['Resume'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.xlsx')

In [85]:
data['Job_Description_processed'] = data['Job Description'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.xlsx')

In [2]:
path_final = "D:/InterviewAutomation/data/final_data.xlsx"

In [3]:
df = pd.read_excel(path_final)

In [ ]:
df.head()

,Unnamed: 0,ID,Name,Role,Transcript,Resume,decision,Reason for decision,Job Description,Transcript_processed,Resume_processed,Job_Description_processed
0,0,brenbr359,brent brown,product manager,product manager interview transcript\n\ninterv...,here's a sample resume for brent brown applyin...,select,experience,we are looking for a skilled product manager w...,product manager interview transcript \n\n inte...,sample resume brent brown applying role produc...,looking skilled product manager expertise data...
1,1,jameay305,james ayala,software engineer,software engineer interview transcript\n\ninte...,here's a sample resume for james ayala applyin...,select,experience,we are looking for a skilled software engineer...,software engineer interview transcript \n\n in...,sample resume james ayala applying software en...,looking skilled software engineer expertise py...
2,2,scotri565,scott rivera,data engineer,here is a simulated interview for scott rivera...,here's a sample resume for scott rivera applyi...,reject,experience,we are looking for a skilled data engineer wit...,simulated interview scott rivera applying role...,sample resume scott rivera applying role data ...,looking skilled data engineer expertise cloud ...
3,3,emilke232,emily kelly,ui engineer,interview transcript: emily kelly for ui engin...,here's a sample resume for emily kelly:\n\nemi...,select,experience,we are looking for a skilled ui engineer with ...,interview transcript emily kelly ui engineer p...,sample resume emily kelly \n\n emily kelly \n ...,looking skilled ui engineer expertise uiux des...
4,4,ashlra638,ashley ray,data scientist,data scientist interview transcript\n\ncompany...,here's a sample resume for ashley ray applying...,reject,cultural fit,we are looking for a skilled data scientist wi...,data scientist interview transcript \n\n compa...,sample resume ashley ray applying data scienti...,looking skilled data scientist expertise sql m...


In [5]:
new_df = df.drop(columns=['Unnamed: 0', 'Transcript' , 'Resume' , 'Job Description'])

In [6]:
new_df

,ID,Name,Role,decision,Reason for decision,Transcript_processed,Resume_processed,Job_Description_processed
0,brenbr359,brent brown,product manager,select,experience,product manager interview transcript \n\n inte...,sample resume brent brown applying role produc...,looking skilled product manager expertise data...
1,jameay305,james ayala,software engineer,select,experience,software engineer interview transcript \n\n in...,sample resume james ayala applying software en...,looking skilled software engineer expertise py...
2,scotri565,scott rivera,data engineer,reject,experience,simulated interview scott rivera applying role...,sample resume scott rivera applying role data ...,looking skilled data engineer expertise cloud ...
3,emilke232,emily kelly,ui engineer,select,experience,interview transcript emily kelly ui engineer p...,sample resume emily kelly \n\n emily kelly \n ...,looking skilled ui engineer expertise uiux des...
4,ashlra638,ashley ray,data scientist,reject,cultural fit,data scientist interview transcript \n\n compa...,sample resume ashley ray applying data scienti...,looking skilled data scientist expertise sql m...
...,...,...,...,...,...,...,...,...
2953,kotasai181,hyma garg,data scientist,select,"excellent communication skills, strong technic...",interviewer dr rachel kim data science lead \n...,detailed dummy resume hyma garg \n\n hyma garg...,expectedexperience 35 year domain data analy...
2954,kotasai183,eshani sharma,data scientist,reject,"low problem-solving abilities, not enough tech...",interview transcript eshani sharma data scie...,detailed dummy resume eshani sharma \n\n eshan...,expectedexperience 02 year domain data analy...
2955,kotasai184,palash kumar,data scientist,reject,"low problem-solving abilities, inability to co...",transcript interview palash kumar data scienti...,palash kumar \n contact information \n\n emai...,expectedexperience 35 year domain data analy...
2956,kotasai186,kashvi das,data scientist,reject,"unsuitable for the job role, not enough techni...",interview transcript kashvi da data scientist ...,detailed dummy resume kashvi da \n\n contact i...,expectedexperience 68 year domain data analy...


In [7]:
new_df['Transcript_words'] = new_df['Transcript_processed'].apply(lambda x: len(x.split())) 
new_df['Transcript_length'] = new_df['Transcript_processed'].apply(lambda x:len(x))

new_df['Resume_words'] = new_df['Resume_processed'].apply(lambda x: len(x.split())) 
new_df['Resume_length'] = new_df['Resume_processed'].apply(lambda x:len(x))

In [8]:
new_df.head()

,ID,Name,Role,decision,Reason for decision,Transcript_processed,Resume_processed,Job_Description_processed,Transcript_words,Transcript_length,Resume_words,Resume_length
0,brenbr359,brent brown,product manager,select,experience,product manager interview transcript \n\n inte...,sample resume brent brown applying role produc...,looking skilled product manager expertise data...,348,2537,221,2040
1,jameay305,james ayala,software engineer,select,experience,software engineer interview transcript \n\n in...,sample resume james ayala applying software en...,looking skilled software engineer expertise py...,387,2974,241,2231
2,scotri565,scott rivera,data engineer,reject,experience,simulated interview scott rivera applying role...,sample resume scott rivera applying role data ...,looking skilled data engineer expertise cloud ...,282,1989,255,2183
3,emilke232,emily kelly,ui engineer,select,experience,interview transcript emily kelly ui engineer p...,sample resume emily kelly \n\n emily kelly \n ...,looking skilled ui engineer expertise uiux des...,402,2836,237,2035
4,ashlra638,ashley ray,data scientist,reject,cultural fit,data scientist interview transcript \n\n compa...,sample resume ashley ray applying data scienti...,looking skilled data scientist expertise sql m...,306,2187,299,2632


In [9]:
avg_words_role = new_df.groupby(['Role','decision'])['Transcript_words'].mean()

In [10]:
avg_words_role

Role               decision
data analyst       reject      376.007194
                   select      440.632812
data engineer      reject      354.907407
                   select      428.108787
data scientist     reject      365.095395
                   select      420.697917
product manager    reject      341.482759
                   select      418.569170
software engineer  reject      346.182143
                   select      400.767528
ui designer        reject      383.600000
                   select      432.403670
ui engineer        reject      314.534722
                   select      374.360544
Name: Transcript_words, dtype: float64

In [12]:
## Statistical Analysis for checking wether there is a significant difference between the number of words in Transcript, for select and reject

#### Null Hypothesis : The mean number of words in transcript for the decision of rejection and selection are equal 
#### Alternative hypothesis : The mean number of words in transcript for the decision of rejection and selection are not equal

In [11]:
new_df.groupby(['decision'])['Transcript_words'].mean()

decision
reject    353.499672
select    415.713589
Name: Transcript_words, dtype: float64

In [12]:
means = new_df.groupby(['decision'])['Transcript_words'].mean().values

In [13]:
reject_sample = new_df[new_df['decision'] == 'reject'].sample(300)
select_sample = new_df[new_df['decision'] == 'select'].sample(300)

In [14]:
reject_var = (reject_sample['Transcript_words'].std())**2 
select_var = (select_sample['Transcript_words'].std())**2

In [15]:
reject_var/select_var  # This is in between 0.5 and 2, so we need to take the variances as almost equal 

0.8821574140776041

In [16]:
# Using T-distribution for testing 
# Sampling the original distribution 

from scipy.stats import ttest_ind

# Perform a two-sample t-test


t_stat, p_value = ttest_ind(
    reject_sample['Transcript_words'], 
    select_sample['Transcript_words'], 
    equal_var=False # Use Welch's t-test if variances are unequal
)

print("T-statistic:", t_stat)
print("P-value:", p_value)

T-statistic: -7.778672124809383
P-value: 3.2450284968961304e-14


In [17]:
## From this we get to know that , there is a significant evidence to reject Null hypothesis that the average number of words for select is equal to the rejection

In [18]:

def T_test(role , reject_sample,select_sample):
    t_stat, p_value = ttest_ind(
        reject_sample['Transcript_words'], 
        select_sample['Transcript_words'], 
        equal_var=False  # Use Welch's t-test if variances are unequal
    )

    if p_value < 0.05 :
        print("Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for the role of {}".format(role))
    else : 
        print("Accepting Null Hypothesis , Mean number of Transcript words for rejection and selection are equal for the role of {}".format(role))

In [19]:
def role_wise_test(roles):
    for role in roles:
        curr_df = new_df[new_df['Role'] == role] 
        reject_sample = curr_df[curr_df['decision'] == 'reject'].sample(100)
        select_sample = curr_df[curr_df['decision'] == 'select'].sample(100) 
        T_test(role , reject_sample , select_sample)

In [20]:
role_wise_test(new_df['Role'].unique())

Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for the role of product manager
Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for the role of software engineer
Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for the role of data engineer
Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for the role of ui engineer
Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for the role of data scientist
Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for the role of data analyst
Rejecting Null Hypothesis , Mean number of Transcript words for rejection and selection are significantly different for 

In [21]:
## From this we know that, for every role there is a significant difference between the average number of words for selection and rejection for every role

In [22]:
new_df.groupby(['Role'])['Resume_words'].mean() # From this the average number of words in resume for every role are almost equal

Role
data analyst         286.385768
data engineer        295.504912
data scientist       290.685811
product manager      273.021401
software engineer    264.444646
ui designer          285.034188
ui engineer          251.989691
Name: Resume_words, dtype: float64

In [23]:

def T_test1(role , reject_sample,select_sample):
    t_stat, p_value = ttest_ind(
        reject_sample['Transcript_length'], 
        select_sample['Transcript_length'], 
        equal_var=False  # Use Welch's t-test if variances are unequal
    )

    if p_value < 0.05 :
        print("Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of {}".format(role))
    else : 
        print("Accepting Null Hypothesis , Mean length of Transcript for rejection and selection are equal for the role of {}".format(role))

In [24]:
new_df.groupby(['Role','decision'])['Transcript_length'].mean()

Role               decision
data analyst       reject      2818.964029
                   select      3456.015625
data engineer      reject      2661.629630
                   select      3280.334728
data scientist     reject      2726.845395
                   select      3253.274306
product manager    reject      2627.183908
                   select      3299.450593
software engineer  reject      2602.950000
                   select      3123.605166
ui designer        reject      2877.280000
                   select      3352.669725
ui engineer        reject      2288.125000
                   select      2807.748299
Name: Transcript_length, dtype: float64

In [25]:
def role_wise_test1(roles):
    for role in roles:
        curr_df = new_df[new_df['Role'] == role] 
        reject_sample = curr_df[curr_df['decision'] == 'reject'].sample(100)
        select_sample = curr_df[curr_df['decision'] == 'select'].sample(100) 
        T_test1(role , reject_sample , select_sample)

In [26]:
role_wise_test1(new_df['Role'].unique())

Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of product manager
Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of software engineer
Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of data engineer
Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of ui engineer
Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of data scientist
Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of data analyst
Rejecting Null Hypothesis , Mean length of Transcript for rejection and selection are significantly different for the role of ui designer


In [28]:
## Finding wether there is a similarity between the two transcripts if so exists, we will drop one of them

In [27]:
## Finding wether there is a similarity between the two transcripts if so exists and we will drop the other transcripts 
## For 90% similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(new_df['Transcript_processed'])

cosine_sim = cosine_similarity(tfidf_matrix)

# Find transcript pairs with similarity >= 0.9
threshold = 0.9
similar_dict = {}

for i in range(cosine_sim.shape[0]):
    similar_indices = []
    for j in range(i + 1, cosine_sim.shape[1]):  # Avoid redundant comparisons
        if cosine_sim[i, j] >= threshold:
            similar_indices.append(j)
    if similar_indices:
        similar_dict[i] = similar_indices

# Print the dictionary
print("Similar Transcripts Dictionary (≥ 90% similarity):")
print(similar_dict)


Similar Transcripts Dictionary (≥ 90% similarity):
{}


In [28]:
## Finding wether there is a similarity between the two transcripts if so exists and we will drop the other transcripts 
## For 80% similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(new_df['Transcript_processed'])

cosine_sim = cosine_similarity(tfidf_matrix)

# Find transcript pairs with similarity >= 0.8
threshold = 0.8
similar_dict = {}

for i in range(cosine_sim.shape[0]):
    similar_indices = []
    for j in range(i + 1, cosine_sim.shape[1]):  # Avoid redundant comparisons
        if cosine_sim[i, j] >= threshold:
            similar_indices.append(j)
    if similar_indices:
        similar_dict[i] = similar_indices

# Print the dictionary
print("Similar Transcripts Dictionary (≥ 80% similarity):")
print(similar_dict)


Similar Transcripts Dictionary (≥ 80% similarity):
{600: [1478], 672: [932], 716: [1080], 745: [1017], 774: [992, 1494], 992: [1494], 1018: [1397], 1026: [2890], 1170: [1389, 1407], 1849: [1972], 1897: [1909], 1968: [1985], 2450: [2514, 2551], 2464: [2597, 2626], 2468: [2626], 2473: [2579], 2476: [2572], 2482: [2535], 2492: [2601], 2497: [2614], 2510: [2589], 2514: [2551], 2519: [2534], 2525: [2541, 2565], 2597: [2626], 2606: [2611], 2659: [2774], 2661: [2778], 2667: [2739], 2673: [2725], 2678: [2728, 2828], 2679: [2760, 2789], 2711: [2754], 2714: [2766, 2805], 2740: [2764, 2849], 2743: [2812], 2750: [2812], 2760: [2789], 2766: [2805], 2767: [2790]}


In [29]:
tfidf_vectorizer1 = TfidfVectorizer()
transcript_matrix = tfidf_vectorizer1.fit_transform(new_df['Transcript_processed'])

tfidf_vectorizer2 = TfidfVectorizer()
resume_matrix = tfidf_vectorizer2.fit_transform(new_df['Resume_processed'])

tfidf_vectorizer3 = TfidfVectorizer()
jobDescription_matrix = tfidf_vectorizer3.fit_transform(new_df['Job_Description_processed'])


In [30]:
# Use a single TfidfVectorizer for both Resume and Job Description
tfidf_vectorizer4 = TfidfVectorizer()
combined_text = new_df['Resume_processed'] + " " + new_df['Job_Description_processed']
tfidf_vectorizer4.fit(combined_text)

# Split the combined TF-IDF matrix back into Resume and Job Description matrices
resume_matrix = tfidf_vectorizer4.transform(new_df['Resume_processed'])
jobDescription_matrix = tfidf_vectorizer4.transform(new_df['Job_Description_processed'])

# Calculate row-wise cosine similarity
cos_rjd = []
for i in range(new_df.shape[0]):
    cos_rjd.append(cosine_similarity(resume_matrix[i], jobDescription_matrix[i])[0][0])


In [31]:
new_df['Resume_JD'] = cos_rjd

In [32]:
# Use a single TfidfVectorizer for both Resume and Transcript
tfidf_vectorizer5 = TfidfVectorizer()
combined_text = new_df['Resume_processed'] + " " + new_df['Transcript_processed']
tfidf_vectorizer5.fit(combined_text)

# Split the combined TF-IDF matrix back into Resume and Job Description matrices
resume_matrix = tfidf_vectorizer5.transform(new_df['Resume_processed'])
Transcript_matrix = tfidf_vectorizer5.transform(new_df['Transcript_processed'])

# Calculate row-wise cosine similarity
cos_rt = []
for i in range(new_df.shape[0]):
    cos_rt.append(cosine_similarity(resume_matrix[i], Transcript_matrix[i])[0][0])

In [33]:
new_df['Resume_Transcript'] = cos_rt

In [34]:
# Use a single TfidfVectorizer for both Job description and Transcript
tfidf_vectorizer6 = TfidfVectorizer()
combined_text = new_df['Job_Description_processed'] + " " + new_df['Transcript_processed']
tfidf_vectorizer6.fit(combined_text)

# Split the combined TF-IDF matrix back into Resume and Job Description matrices
JD_matrix = tfidf_vectorizer6.transform(new_df['Job_Description_processed'])
Transcript_matrix = tfidf_vectorizer6.transform(new_df['Transcript_processed'])

# Calculate row-wise cosine similarity
cos_jdt = []
for i in range(new_df.shape[0]):
    cos_jdt.append(cosine_similarity(JD_matrix[i], Transcript_matrix[i])[0][0])

In [35]:
new_df['JobDescription_Transcript'] = cos_jdt

In [36]:
new_df.head()

,ID,Name,Role,decision,Reason for decision,Transcript_processed,Resume_processed,Job_Description_processed,Transcript_words,Transcript_length,Resume_words,Resume_length,Resume_JD,Resume_Transcript,JobDescription_Transcript
0,brenbr359,brent brown,product manager,select,experience,product manager interview transcript \n\n inte...,sample resume brent brown applying role produc...,looking skilled product manager expertise data...,348,2537,221,2040,0.351376,0.389509,0.151835
1,jameay305,james ayala,software engineer,select,experience,software engineer interview transcript \n\n in...,sample resume james ayala applying software en...,looking skilled software engineer expertise py...,387,2974,241,2231,0.272022,0.380939,0.091687
2,scotri565,scott rivera,data engineer,reject,experience,simulated interview scott rivera applying role...,sample resume scott rivera applying role data ...,looking skilled data engineer expertise cloud ...,282,1989,255,2183,0.465514,0.384318,0.077372
3,emilke232,emily kelly,ui engineer,select,experience,interview transcript emily kelly ui engineer p...,sample resume emily kelly \n\n emily kelly \n ...,looking skilled ui engineer expertise uiux des...,402,2836,237,2035,0.313523,0.381142,0.190069
4,ashlra638,ashley ray,data scientist,reject,cultural fit,data scientist interview transcript \n\n compa...,sample resume ashley ray applying data scienti...,looking skilled data scientist expertise sql m...,306,2187,299,2632,0.292945,0.334349,0.072062


In [37]:
new_df.groupby(['decision'])['Resume_Transcript'].mean()

decision
reject    0.403343
select    0.385507
Name: Resume_Transcript, dtype: float64

In [38]:
new_df.groupby(['decision'])['Resume_JD'].mean()

decision
reject    0.223339
select    0.219486
Name: Resume_JD, dtype: float64

In [39]:
new_df.groupby(['decision'])['JobDescription_Transcript'].mean()

decision
reject    0.096208
select    0.116458
Name: JobDescription_Transcript, dtype: float64

In [40]:
import gensim
from gensim.models import Word2Vec

# Preprocess the transcripts
def preprocess_text(text):
    # Tokenize text
    tokens = text.split()
    return tokens

# Preprocess all transcripts
processed_transcripts = [preprocess_text(transcript) for transcript in new_df['Transcript_processed']]

In [41]:
# Train Word2Vec model
model = Word2Vec(sentences=processed_transcripts, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for future use



In [42]:
model.save("word2vec_transcript_model")

In [43]:
# Example: Vectorizing a single transcript
def vectorize_transcript(transcript, model):
    tokens = preprocess_text(transcript)
    # Get the word vectors for each word in the transcript and average them
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]
    if word_vectors:
        return sum(word_vectors) / len(word_vectors)
    else:
        return None  # If no words found in the model


# Example: You can also vectorize all transcripts at once
Transcript_vectors = np.array([vectorize_transcript(transcript, model) for transcript in new_df['Transcript_processed']])


In [44]:
processed_resumes =  [preprocess_text(resume) for resume in new_df['Resume_processed']]
processed_JD =  [preprocess_text(jd) for jd in new_df['Job_Description_processed']]

In [45]:
model1 =  Word2Vec(sentences=processed_resumes, vector_size=100, window=5, min_count=1, workers=4)
model2 = Word2Vec(sentences=processed_JD, vector_size=100, window=5, min_count=1,workers=4)

In [46]:
Resume_vectors = np.array([vectorize_transcript(resume, model1) for resume in new_df['Resume_processed']])
JD_vectors = np.array([vectorize_transcript(jd, model2) for jd in new_df['Job_Description_processed']])

In [58]:
model1.save("word2vec_resume_model")
model2.save("word2vec_job_description_model")

In [47]:
re_tr_simi_w2v = [] 
re_jd_simi_w2v = []
tr_jd_simi_w2v = [] 

for i in range(new_df.shape[0]):
    re_tr_simi_w2v.append(cosine_similarity(Resume_vectors[i].reshape(-1,1),Transcript_vectors[i].reshape(-1,1))[0][0])
    re_jd_simi_w2v.append(cosine_similarity(Resume_vectors[i].reshape(-1,1),JD_vectors[i].reshape(-1,1))[0][0]) 
    tr_jd_simi_w2v.append(cosine_similarity(Transcript_vectors[i].reshape(-1,1),JD_vectors[i].reshape(-1,1))[0][0]) 

In [48]:
new_df['Resume_Transcript_W2V'] = re_tr_simi_w2v 
new_df['Resume_JD_W2V'] = re_jd_simi_w2v 
new_df['JobDescription_Transcript_W2V'] = tr_jd_simi_w2v 

In [49]:
new_df.groupby(['decision'])['Resume_Transcript_W2V'].mean()

decision
reject   -0.352594
select   -0.120557
Name: Resume_Transcript_W2V, dtype: float32

In [50]:
new_df.groupby(['decision'])['Resume_JD_W2V'].mean()

decision
reject    0.248851
select    0.344948
Name: Resume_JD_W2V, dtype: float32

In [51]:
new_df.groupby(['decision'])['JobDescription_Transcript_W2V'].mean()

decision
reject   -0.294813
select   -0.190244
Name: JobDescription_Transcript_W2V, dtype: float32

In [52]:
from transformers import AutoTokenizer, AutoModel
import torch

d:\InterviewAutomation\Automation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"  # You can replace this with other models
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


ImportError: Can't connect to HTTPS URL because the SSL module is not available.